<a href="https://colab.research.google.com/github/mechanic3000/GB_algo_data/blob/master/course_work_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Предсказать вероятность того, подойдет ли репетитор для подготовки к экзамену по математике

In [43]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from matplotlib import pyplot as plt



In [79]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GB_data_analys_algo/gb-choose-tutors/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GB_data_analys_algo/gb-choose-tutors/test.csv')

In [45]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [46]:
def get_error(pred, y):
    return sum(pred != y) / len(y)

In [132]:
def adaboost(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    # Начальные веса объектов
    w = np.ones(n_objects) / n_objects

    # Деревья с весами будем записывать в список
    models = []

    for n in range(N):
        # Зададим дерево и обучим его
        clf = DecisionTreeClassifier(max_depth=5)
        clf.fit(X, y, sample_weight=w)

        predictions = clf.predict(X)
        error = get_error(predictions, y)
        
        # отбросим дерево, если его ошибка больше 0.5
        # Запишем условие в общем виде (применимо к небинарным классификаторам)
        if error >= (1 - 1/n_classes): 
            continue

        # Обработаем граничные значения ошибок
        if error == 0:
            error += 1e-10
   
        # Вычислим вес для дерева
        alpha = 0.5 * np.log((1 - error) / error)


        # Найдем индексы правильно классифицированных элементов
        wrong_mask = predictions != y

        # Увеличим веса для неправильно классифицированных элементов
        w[wrong_mask] *= np.exp(alpha)
        # Уменьшаем веса для правильно классифицированных элементов
        w[~wrong_mask] *= np.exp(-alpha)

        # Нормализуем веса
        w /= w.sum()

        # Добавим дерево с весом в список
        models.append((alpha, clf))
    
    return models

In [48]:
def predict(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        # Для каждого предсказания будем прибавлять alpha к
        # элементу с индексом предсказанного класса
        y_pred[range(n_objects), prediction] += alpha
    
    # выберем индексы с максимальными суммарными весами -
    # получим предсказанные алгоритмом классы
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

In [49]:
train_df.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


In [50]:
train_df.shape, test_df.shape

((10000, 13), (10000, 12))

In [51]:
train_df.columns

Index(['Id', 'age', 'years_of_experience', 'lesson_price', 'qualification',
       'physics', 'chemistry', 'biology', 'english', 'geography', 'history',
       'mean_exam_points', 'choose'],
      dtype='object')

In [52]:
train_df.describe()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,4999.50000,45.800900,1.974800,1702.440000,1.724300,0.37060,0.121500,0.117200,0.059100,0.02770,0.018000,64.435200,0.110900
std,2886.89568,8.030274,1.766883,523.789062,0.798845,0.48299,0.326724,0.321675,0.235824,0.16412,0.132958,13.595024,0.314024
min,0.00000,23.000000,0.000000,200.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,33.000000,0.000000
25%,2499.75000,40.000000,0.000000,1300.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,55.000000,0.000000
50%,4999.50000,46.000000,2.000000,1550.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,63.000000,0.000000
75%,7499.25000,51.000000,3.000000,2150.000000,2.000000,1.00000,0.000000,0.000000,0.000000,0.00000,0.000000,74.000000,0.000000
max,9999.00000,68.000000,9.000000,3950.000000,4.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,100.000000,1.000000


In [64]:
train_df.describe()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,4999.50000,45.800900,1.974800,1702.440000,1.724300,0.37060,0.121500,0.117200,0.059100,0.02770,0.018000,64.435200,0.110900
std,2886.89568,8.030274,1.766883,523.789062,0.798845,0.48299,0.326724,0.321675,0.235824,0.16412,0.132958,13.595024,0.314024
min,0.00000,23.000000,0.000000,200.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,33.000000,0.000000
25%,2499.75000,40.000000,0.000000,1300.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,55.000000,0.000000
50%,4999.50000,46.000000,2.000000,1550.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,63.000000,0.000000
75%,7499.25000,51.000000,3.000000,2150.000000,2.000000,1.00000,0.000000,0.000000,0.000000,0.00000,0.000000,74.000000,0.000000
max,9999.00000,68.000000,9.000000,3950.000000,4.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,100.000000,1.000000


In [90]:
train_df.loc[train_df['age'] < 40, 'age_cat'] = 1
train_df.loc[(train_df['age'] >= 40) & (train_df['age'] < 50) , 'age_cat'] = 2
train_df.loc[(train_df['age'] >= 50) & (train_df['age'] < 60) , 'age_cat'] = 3
train_df.loc[train_df['age'] >= 60, 'age_cat'] = 3

train_df['age_cat'].isna().sum()

0

In [101]:
train_df.loc[train_df['lesson_price'] < 1300, 'lesson_price_cat'] = 1
train_df.loc[(train_df['lesson_price'] >= 1300) & (train_df['lesson_price'] < 1600) , 'lesson_price_cat'] = 2
train_df.loc[(train_df['lesson_price'] >= 1600) & (train_df['lesson_price'] < 2200) , 'lesson_price_cat'] = 3
train_df.loc[train_df['lesson_price'] >= 2200, 'lesson_price_cat'] = 3

train_df['lesson_price_cat'].isna().sum()

0

In [106]:
train_df.loc[train_df['mean_exam_points'] < 55, 'mean_exam_points_cat'] = 1
train_df.loc[(train_df['mean_exam_points'] >= 55) & (train_df['mean_exam_points'] < 65) , 'mean_exam_points_cat'] = 2
train_df.loc[(train_df['mean_exam_points'] >= 65) & (train_df['mean_exam_points'] < 74) , 'mean_exam_points_cat'] = 3
train_df.loc[train_df['mean_exam_points'] >= 74, 'mean_exam_points_cat'] = 3

train_df['mean_exam_points_cat'].isna().sum()

0

In [54]:
scaling_features = ['age', 'mean_exam_points']

In [126]:
features = [
'age_cat', 
'years_of_experience', 
'lesson_price_cat', 
'qualification',
'physics', 
'chemistry', 
'biology', 
'english', 
'geography', 
'history',
'mean_exam_points_cat'
]

target = ['choose']



```
Id - айди
age - возраст преподавателя
years_of_experience - стаж преподавателя
lesson_price - стоимость урока
qualification - квалификация преподавателя
physics - преподает ли физику
chemistry - преподает ли химию
biology - преподает ли биологию
english - преподает ли английский
geography - преподает ли географию
history - преподает ли историю
mean_exam_points - средний балл за экзамен учеников
choose - целевое значени, выбирать или нет преподавателя
```



In [127]:
X_train, X_test, y_train, y_test = train_test_split(train_df[features].values, 
                                                                    np.squeeze(train_df[target].values),
                                                                    test_size=0.3, 
                                                                    random_state=42)

In [133]:
N = 35

models = adaboost(X_train, y_train, N)

In [134]:
print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(X_train, models), y_train)) * 100:.3f}')
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 88.986
Точность алгоритма на тестовой выборке: 87.933


**Сохранение результата**

In [59]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GB_data_analys_algo/gb-choose-tutors/sample_submission.csv')

In [112]:
test_df.loc[test_df['age'] < 40, 'age_cat'] = 1
test_df.loc[(test_df['age'] >= 40) & (test_df['age'] < 50) , 'age_cat'] = 2
test_df.loc[(test_df['age'] >= 50) & (test_df['age'] < 60) , 'age_cat'] = 3
test_df.loc[test_df['age'] >= 60, 'age_cat'] = 3

test_df.loc[test_df['lesson_price'] < 1300, 'lesson_price_cat'] = 1
test_df.loc[(test_df['lesson_price'] >= 1300) & (test_df['lesson_price'] < 1600) , 'lesson_price_cat'] = 2
test_df.loc[(test_df['lesson_price'] >= 1600) & (test_df['lesson_price'] < 2200) , 'lesson_price_cat'] = 3
test_df.loc[test_df['lesson_price'] >= 2200, 'lesson_price_cat'] = 3

test_df.loc[test_df['mean_exam_points'] < 55, 'mean_exam_points_cat'] = 1
test_df.loc[(test_df['mean_exam_points'] >= 55) & (test_df['mean_exam_points'] < 65) , 'mean_exam_points_cat'] = 2
test_df.loc[(test_df['mean_exam_points'] >= 65) & (test_df['mean_exam_points'] < 74) , 'mean_exam_points_cat'] = 3
test_df.loc[test_df['mean_exam_points'] >= 74, 'mean_exam_points_cat'] = 3

test_df['age_cat'].isna().sum(),
test_df['lesson_price_cat'].isna().sum(),
test_df['mean_exam_points_cat'].isna().sum()

0

In [135]:
test_predict = predict(test_df[features].values, models)

In [136]:
submit['choose'] = test_predict
submit['choose'].unique()

array([0, 1])

In [137]:
submit.to_csv('cf_submit.csv', index=False)

[www.kaggle.com](https://www.kaggle.com/c/gb-choose-tutors/leaderboard)

0.76262